In [1]:
import pandas as pd
import numpy as np
import os
from keras.preprocessing.image import Iterator,load_img
from keras import layers
from keras import models
from keras import optimizers
import math 
import matplotlib.pyplot as plt
import PIL
from keras.preprocessing.image import ImageDataGenerator
from fast_ml.model_development import train_valid_test_split


In [2]:
dirname = os.path.abspath('data/uw-madison-gi-tract-image-segmentation')
print(dirname)

C:\Users\catia\UWMadisonGITractImageSegmentation\data\uw-madison-gi-tract-image-segmentation


In [15]:
class TrainIterator():
    """Class to iterate A and B images at the same time."""
    
    def __init__(
                 self, 
                 dir_dataset, 
                 csv_t_filename_prefix = "train.csv", 
                 dir_Training = "train",
                 target_size=(150,150),
                 batch_size=128, 
                 shuffle=True, 
                 seed = None,
                ):
    
    
        self.target_size=target_size
        
        ## Training
        self.path_Training = os.path.join(dir_dataset, dir_Training)
        
        self.ficheiros_Training = set(x for x in os.listdir(self.path_Training))
        print ("Numero de ficheiros Training:", len(self.ficheiros_Training))
        
        self.n=len(self.ficheiros_Training)
        
        self.a_fnames = sorted(os.listdir(self.path_Training))

        self.filenames=list(self.ficheiros_Training)       
        # Carega ficheiro CSV com posicao da Fove
        ## Validation
        
        
        ## Training
        
        path_csv_t = os.path.join(dir_dataset,csv_t_filename_prefix)

        self.data_Training = pd.read_csv(path_csv_t)
        

        self.ID_training=self.data_Training.iloc[:,0]


        self.Class_training=self.data_Training.iloc[:,1]
        self.Segmentation_training=self.data_Training.iloc[:,2]
        #print(ID_training.head())
        #print(Class_training.head())
        #print(Segmentation_training.head())
        #print(len(ID_training))
        
        self.img_shape_a = self._get_img_shape(self.target_size, 1)
        self.img_shape_b = self._get_img_shape(self.target_size, 1)
        
        
        
        super(TrainIterator, self).__init__()  

    def load_imgs(self, idx):
        """Get a pair of images with index idx."""
        a_fname = self.filenames[idx]

        a = load_img(os.path.join(self.path_Training, a_fname),
                     grayscale=True,
                     target_size=self.target_size)

        a = np.array(a)

        if(len(a.shape)<3):
            a = a.reshape(self.img_shape_a)

        return a
            
    def _get_img_shape(self, size, ch=1):

        img_shape = size + (ch,)

        return img_shape
    def load_to_mem(self):
        """Load images to memory."""
        self.a = np.zeros((self.N,) + self.img_shape_a)
    
        for idx in range(self.N):
            ai = self.load_imgs(idx)
            self.a[idx] = ai 


    def next(index_generator):
        """Get the next pair of the sequence."""

        # Lock the iterator when the index is changed.
        #lock = threading.Lock()
        #with lock:
        index_array = next(self.index_generator)    
        current_batch_size = len(index_array)

      
        #Cria as varaiveis batch
        batch_X = np.zeros((current_batch_size,) + self.img_shape_a)
        batch_Y = np.zeros((current_batch_size,2))
        
        for i, j in enumerate(index_array):
            a_img= self.load_imgs(j)
            
            a_img = (a_img - a_img.mean()) / (a_img.std())
            #if(np.max(batch_a)>1.):
                #a_img=a_img/255.
           
            batch_X[i] = a_img
            batch_Y[i] = np.array([self.ID_training[j], 
                                   self.Class_training[j],self.Segmentation_training[j]])
 
        
        return [batch_X,batch_Y]

    def generator():
        while 1:
            x,y = next()
            x=x/255
            #print x.shape,y.shape
            yield x,y
            

In [16]:
train_it=TrainIterator(dirname)


Numero de ficheiros Training: 85


In [17]:
from keras.layers import Input,Dense,Flatten,MaxPooling2D
from keras.layers.core import Activation, Dropout
from keras.layers.convolutional import Conv2D
from tensorflow.keras.layers import BatchNormalization
from keras.models import Model

import numpy as np
from keras import regularizers

In [18]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150,1)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3),kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3),kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3),kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(2))

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      320       
                                                                 
 batch_normalization (BatchN  (None, 148, 148, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 72, 72, 64)       256       
 hNormalization)                                                 
                                                        

In [20]:
model.compile(loss='mse',
              optimizer='RMSProp',
              metrics=['mae'])

In [27]:
num_epochs = 25
all_mae_histories = []

history=model.fit_generator(train_it,
      steps_per_epoch=5,
      epochs=25)

mae_history = history.history['val_mean_absolute_error']
all_mae_histories.append(mae_history)

ValueError: Failed to find data adapter that can handle input: <class '__main__.TrainIterator'>, <class 'NoneType'>

In [22]:
model.save('new_model.h5')

In [23]:
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(2,len(loss)-4)

def smooth_curve(points, factor=0.8):
  smoothed_points = []
  for point in points:
    if smoothed_points:
      previous = smoothed_points[-1]
      smoothed_points.append(previous * factor + point * (1 - factor))
    else:
      smoothed_points.append(point)
  return smoothed_points

plt.plot(epochs,
         smooth_curve(loss[2:21]), 'bo', label='Smoothed training loss')
plt.plot(epochs,
         smooth_curve(val_loss[2:21]), 'b', label='Smoothed validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.figure()

NameError: name 'history' is not defined

In [132]:
acc = history.history['acc']
val_acc = history.history['val_acc']

epochs = range(2,len(loss)-4)

def smooth_curve(points, factor=0.8):
  smoothed_points = []
  for point in points:
    if smoothed_points:
      previous = smoothed_points[-1]
      smoothed_points.append(previous * factor + point * (1 - factor))
    else:
      smoothed_points.append(point)
  return smoothed_points

plt.plot(epochs,
         smooth_curve(acc[2:21]), 'bo', label='Smoothed training accuracy')
plt.plot(epochs,
         smooth_curve(val_acc[2:21]), 'b', label='Smoothed validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

NameError: name 'history' is not defined

In [ ]:
mae = history.history['mean_absolute_error']
val_mae = history.history['val_mean_absolute_error']

epochs = range(2,len(loss)-4)

plt.plot(epochs, smooth_curve(mae[2:21]), 'bo', label='MAE')
plt.plot(epochs, smooth_curve(val_loss[2:21]), 'b', label='Validation MAE')
plt.title('Training and validation MAE')
plt.legend()

plt.show()

In [ ]:
average_mae_history = [np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

In [ ]:
plt.clf()

smooth_mae_history = smooth_curve(average_mae_history[2:21])

plt.plot(range(1, len(smooth_mae_history)+1), smooth_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

In [ ]:
predictions = model.predict(test_it.load_to_mem(),steps=10)

In [ ]:
predictions

NameError: name 'train_ds' is not defined